# NER

Распознавание именованных сущностей (Named-entity recognition, [NER](https://en.wikipedia.org/wiki/Named-entity_recognition)) иначе известное как классификация токенов, разделение сущностей или извлечение сущностей ) — это подзадача извлечения информации, которая направлена ​​на поиск и классификацию именованных сущностей, упомянутых в неструктурированном тексте заранее определенных категорий, такие как человек имена, организации, местоположения, медицинские коды , выражения времени, количества, денежные значения, проценты и т.д..

Этот ноутбук разработан на основе статьи [Ner AutoML](https://habr.com/ru/company/vtb/blog/651525/) и открытых наборов данных [[1](http://labinform.ru/pub/named_entities/descr_ne.htm)], [[2](http://labinform.ru/pub/named_entities/descr_ne.htm)], [[3](https://github.com/dialogue-evaluation/factRuEval-2016/)].

<img src="https://habrastorage.org/r/w1560/getpro/habr/upload_files/b39/83d/927/b3983d92718f8fb4686aa679866513ef.jpeg" width="500" align="center"/>


#### Установка зависимостей

Для начала нам необходимо определиться с форматом, в котором мы будем хранить данные с разметкой.


Формат jsonlines хранения каждого текста удобен, поскольку многие датасеты и [средства аннотации](https://labelstud.io/playground/) его придерживаются:

<div>
<img src="https://dl.uploadgram.me/62a74788afb75h?raw" width="400"/>
</div>

In [ ]:
!pip install -q transformers datasets jsonlines seqeval

     |████████████████████████████████| 5.8 MB 28.0 MB/s 
     |████████████████████████████████| 451 kB 76.8 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 7.6 MB 68.3 MB/s 
     |████████████████████████████████| 182 kB 79.4 MB/s 
     |████████████████████████████████| 212 kB 75.3 MB/s 
     |████████████████████████████████| 132 kB 83.5 MB/s 
     |████████████████████████████████| 127 kB 72.2 MB/s 


## Импорты

### Вариант загрузки модели 1

Открываем [страницу модели](https://huggingface.co/sberbank-ai/sbert_large_nlu_ru/tree/main), но можно выбрать любую [другую](https://huggingface.co/models?language=ru&sort=downloads). Выбираем use in transformers.

In [ ]:
!git lfs install
!git clone https://huggingface.co/sberbank-ai/sbert_large_nlu_ru

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 'sbert_large_nlu_ru'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 28 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (28/28), done.
tcmalloc: large alloc 1471086592 bytes == 0x561a32dbe000 @  0x7fa0b40f32a4 0x5619f799578f 0x5619f79728db 0x5619f79275b3 0x5619f78cb34a 0x5619f78cb806 0x5619f78e8ad1 0x5619f78e9069 0x5619f78e9593 0x5619f798e482 0x5619f782ecc2 0x5619f7815a75 0x5619f7816735 0x5619f781573a 0x7fa0b343ac87 0x5619f781578a
tcmalloc: large alloc 2206621696 bytes == 0x561a8a8ae000 @  0x7fa0b40f32a4 0x5619f799578f 0x5619f79728db 0x5619f79275b3 0x5619f78cb34a 0x5619f78cb806 0x5619f78e8ad1 0x5619f78e9069 0x5619f78e9593 0x5619f798e482 0x5619f782ecc2 0x5619f7815a75 0x5619f7816735 0x5619f781573a 

Внутри каждой модели должны быть файлы:

* config.json — описание архитектуры модели
* pytorch_model.bin — веса модели
* vocab.txt — словарь токенизатора

In [ ]:
!ls /content/sbert_large_nlu_ru

config.json	    pytorch_model.bin  special_tokens_map.json	vocab.txt
flax_model.msgpack  README.md	       tokenizer_config.json


In [ ]:
# функции для анализа автоматической инициализации архитектуры по модели
# и добавления линейного слоя под нужную задачу
from transformers import AutoConfig, AutoModelForTokenClassification

# кол-во лейблов == кол-ву классов меток
config = AutoConfig.from_pretrained('sbert_large_nlu_ru/config.json', num_labels=5)
model = AutoModelForTokenClassification.from_pretrained('sbert_large_nlu_ru/pytorch_model.bin', config=config)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at sbert_large_nlu_ru/pytorch_model.bin and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Вид линейного слоя над головой берта:

<div>
<img src="https://habrastorage.org/r/w1560/getpro/habr/upload_files/7b9/383/298/7b938329885cee3a076db5dff67f58e0.png" width="300"/>
</div>

In [ ]:
# токенизатор конкретной модели
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast(vocab_file='sbert_large_nlu_ru/vocab.txt', do_lower_case=False)

### Вариант загрузки модели 2

In [ ]:
from transformers import BertTokenizerFast
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

### Вид входных данных

Во время обучения мы ожидаем на вход модели 3 тензора:
1. input_ids — входные последовательности 
идентификаторов токенов.
2. attention_mask — соответствующие идентификаторам токенов attention маски. Нуль ставится на те позиции, на которых во входной последовательности находятся [PAD]-токены, дополняющие её до нужной длины. На остальных позициях ставятся единицы.
3. labels — идентификаторы классов именованных сущностей.

На этапе инференса модель принимает на вход только тензор input_ids.


Пример батча для двух последовательностей токенов длины 11:

<div>
<img src="https://habrastorage.org/getpro/habr/upload_files/616/f75/b1a/616f75b1a8131da165613d000f26a1f3.JPG" width="800"/>
</div>



Токенизатор возвращает объект encoding, у которого есть свойства tokens и ids, содержащие соответственно последовательность токенов и их идентификаторов.

In [ ]:
text = 'Сегодня    утром Иван        Иванович решил   зайти  в кафе   Круассан, чтобы позавтракать'

encoding = tokenizer(text,  add_special_tokens=False)[0]
for i, (token, token_id) in enumerate(zip(encoding.tokens, encoding.ids)):
    print((i, token, token_id))

(0, 'Сегодня', 3855)
(1, 'утром', 5111)
(2, 'Иван', 5469)
(3, 'Иванович', 9796)
(4, 'решил', 3636)
(5, 'зайти', 20717)
(6, 'в', 113)
(7, 'кафе', 5558)
(8, 'Кру', 8792)
(9, '##ас', 15060)
(10, '##сан', 7480)
(11, ',', 121)
(12, 'чтобы', 1015)
(13, 'позавтракать', 90384)


В объекте encoding есть два метода char_to_token и token_to_chars.

- Первый метод получает на вход позицию символа в тексте, например начальную позицию сущности, и возвращает номер токена в списке encoding.tokens (или encoding.ids). Этот метод позволяет преобразовать формат jsonline в формат BIO.

- Второй метод получает на вход номер токена из списка encoding.tokens (или encoding.ids), например номер первого токена, который соответствует именованной сущности, и возвращает начальную и конечную позицию этого токена в исходном тексте. Метод позволяет сопоставить спрогнозированные сущности с их позициями в тексте.



<div>
<img src="https://habrastorage.org/getpro/habr/upload_files/
da4/7af/a0e/da47afa0ea60fa5b940a8e38b9da5f1f.JPG" width="800"/>
</div>



In [ ]:
data = {
    'text': 'Сегодня    утром Иван        Иванович решил   зайти  в кафе   Круассан, чтобы позавтракать',
    'entities' : [
        {"label": "person", "start": 17, "end": 37},
        {"label": "location", "start": 62, "end": 70}
    ]
}
 
encoding = tokenizer(data['text'],  add_special_tokens=False)[0]
 
start_token_pos = encoding.char_to_token(data['entities'][1]['start'])
end_token_pos = encoding.char_to_token(data['entities'][1]['end'] - 1)
all_pos = list(range(start_token_pos, end_token_pos + 1))
print('Номера всех токенов именованной сущности:', all_pos)
 
start, _ = encoding.token_to_chars(start_token_pos)
_, end = encoding.token_to_chars(end_token_pos)
print('Начальная и конечная позиция именованной сущности в тексте:', [start, end])

Номера всех токенов именованной сущности: [8, 9, 10]
Начальная и конечная позиция именованной сущности в тексте: [62, 70]


## Устройство конвейера

### TokenizedDataSet

TokenizedDataSet наследуется от класса Dataset и переопределяет методы init, len и getitem. Экземпляр этого класса хранит выборку в поле sequence в таком же формате, который подаётся на вход модели.

In [ ]:
from torch.utils.data import Dataset
 

class TokenizedDataset(Dataset):
    # В конструктор передаем выборку с пердложениями
    def __init__(self, sequences):
        self.sequences = sequences
 
    # Получаем размер выборки
    def __len__(self):
        return len(self.sequences['input_ids'])

    # получаем элемент выборки по индексу
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.sequences.items()}

### Preprocessor

![](https://habrastorage.org/getpro/habr/upload_files/6fb/928/791/6fb928791b6ddbbdce049ce80d16df1d.JPG)

![](https://habrastorage.org/getpro/habr/upload_files/a7f/1a6/bee/a7f1a6bee3a7a9b34871dce11fd6f97b.JPG)

In [ ]:
import os
import jsonlines
import torch

class Preprocessor:
    '''
    Класс для обработки текстов, хранит:
    tokenizer: токенизатор
    label_to_id: словарь отображения меток в их идентификаторы
    id_to_label: словарь отображения идентификаторов в метки
    tensor_names: имена входных тензоров для модели
    max_len: максимальную длину последовательности
    '''
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.label_to_id = {self.tokenizer.pad_token: self.tokenizer.pad_token_id}
        self.id_to_label = {self.tokenizer.pad_token_id: self.tokenizer.pad_token}
        self.tensor_names = ('input_ids', 'attention_mask', 'labels')
        self.max_len = 511 # 0я позаиция заложена для ['CLS'] токена

    '''
    Метод конвертации json в BIO формат.
    json_line: словарь содержащий текст и список именованных сущностей,
    каждый элемент - словарь с именем сущности и позициям начала и конца в тексте
    '''
    def tokenize_text(self, json_line):
        # выделяем сам текст
        text = json_line['text']
        # и его сущности
        entities = json_line['entities']
 
        # применяем токенизатор к тексту, получаем encoding со списком токенов и
        # их идентификаторов
        encoding = self.tokenizer(text, add_special_tokens=False)[0]
        # список для установки токенам их метки, далее заменим "O" на нужную сущность
        # при пересечении ему соответсвует несколько меток
        labels_names = [{'O'} for i in range(len(encoding.tokens))]

        # для каждой сущности из списка именнованых сущностей
        for entity in entities:
            # начальная позиция передается в chat_to_token, чтоб определить
            # start_token_pos - индекс первого токена сущности в encoding.ids
            try:
                start_token_pos = encoding.char_to_token(entity['start'])
                # end_token_pos - индекс последнего токена сущности в encoding.ids
                end_token_pos = encoding.char_to_token(entity['end'] - 1) + 1
            except TypeError:
                print('error with entity:', entity)
                continue
            # имя сущности
            label_name = entity['label']
            # в найденном диапазоне сущности
            for pos in range(start_token_pos, end_token_pos):
                # B = Begin, начало сущности I = Inside, остальная часть сущности
                token_label = f'B-{label_name}' if pos == start_token_pos else f'I-{label_name}'
                # добавляем сущность в множество на соответсвующей позиции label_names
                labels_names[pos].discard('O')
                labels_names[pos].add(token_label)
        # если в списке labels_names несколько меток - конкатенируем, например B-PER|I-LOC
        # на основе подхода, описанного в https://arxiv.org/abs/1908.06926v1
        labels_names = list(map(lambda x: '|'.join(sorted(x)), labels_names))

        # добавляем метки в словари label_to_id и id_to_label
        labels = []
        for label_name in labels_names:
            if label_name not in self.label_to_id:
                self.label_to_id[label_name] = self.tokenizer.pad_token_id + len(self.label_to_id)
                self.id_to_label[self.label_to_id[label_name]] = label_name
            labels.append(self.label_to_id[label_name])
        # список токенов и идентификаторов, id-шники лейблов
        return encoding, labels

    # метод для токенизации всех текстов в заданном файле
    def tokenize_texts(self, file_path):
 
        encodings = []
        labels = []
        # открываем файл
        with open(file_path, mode='r') as f:
            for json_line in f.readlines():
                json_line = eval(json_line)
                # получаем энкодинги и id-шники лейблов
                e, l = self.tokenize_text(json_line)
                encodings.append(e)
                labels.append(l)
        
        return encodings, labels

    # функция для дополнения спецтокенами
    # array: последовательность спецтокенов
    # type_: input_ids, attention_mask или labels
    def add_special_tokens(self, array, type_='input_ids'):
        # Id токенов CLS и PAD
        CLS_ID = self.tokenizer.cls_token_id
        PAD_ID = self.tokenizer.pad_token_id
        # размер паддинга (кол-во слов)
        pad_len = self.max_len - len(array)
        # в зависимости от типа последовательности дополняем спецтокенами
        if type_ == 'input_ids':
            new_array = [CLS_ID] + array + [PAD_ID] * pad_len
        elif type_ == 'attention_mask':
            new_array = [1] + array + [PAD_ID] * pad_len
        elif type_ == 'labels':
            l_id = self.label_to_id['O']
            new_array = [l_id] + array + [PAD_ID] * pad_len
 
        return new_array

    # метод разбиения последловательности больше max_length
    # объект encodings - объект в результате обработки одного текста
    # labels - список идентификаторов меток именованных сущностей
    def cut_long_sequence(self, encoding, labels=None):
        # если разметка отсутсвует, то дробить надо по символам конца предложения
        end_symbols = frozenset(('.', '?', '!'))
        # для проверки на конец предложения 
        split_labels_types = frozenset(('O', 'B'))
        # мы будем передвигать границу, если последовательность должен max_len
        border = self.max_len
        # максимально близкий к border симмвол разделения предложения
        split_position = None
        # если препинаний нету и split_position == None, используем максимально близкий
        # к border индекс токена, который начинается не с ## , чтобы не разделять
        # текст на середине слова
        split_position_alt = None
        split_positions = [0]
        # все последовательности
        sequences = {t: [] for t in self.tensor_names}
        sequence_len = len(encoding.ids)

        # если длина последовательности не превышает максимальную длину, то
        # дополняем специальными токенами
        if sequence_len < self.max_len:
            # дополняем спецтокенами
            s_input_ids = self.add_special_tokens(encoding.ids, 'input_ids')
            sequences['input_ids'].append(s_input_ids)
            if labels is not None:
                # спецтокены для массок аттеншена
                s_attention_mask = self.add_special_tokens(encoding.attention_mask, 'attention_mask')
                sequences['attention_mask'].append(s_attention_mask)
                # спецтокены для лейблов
                s_labels = self.add_special_tokens(labels, 'labels')
                sequences['labels'].append(s_labels)
        
        # иначе разбиваем на последовательности меньше
        else:
            # идем по всей последовательности
            for i in range(sequence_len):
                # если не вышли за пределы границы
                if i < border:
                    # если есть разметка
                    if labels is not None:
                        # если символы конца предложения не являются сущностью
                        if encoding.tokens[i] in end_symbols and self.id_to_label[labels[i]] == 'O':
                            # устанавливаем индекс разбиения на него
                            split_position = i + 1
                        # иначе выбираем алтернативный ближайший токен не сущность
                        if i + 1 < sequence_len:
                            if encoding.tokens[i + 1][:2] != '##' and self.id_to_label[labels[i + 1]][0] in split_labels_types:
                                split_position_alt = i + 1
                    # если разметки нету
                    else:
                        # если это конец предложения ставим разбиение
                        if encoding.tokens[i] in end_symbols:
                            split_position = i + 1
                        # иначе на ближайший токен (не из середины слова)
                        if i + 1 < sequence_len:
                            if encoding.tokens[i + 1][:2] != '##':
                                split_position_alt = i + 1
                # если вышли за пределы границы
                else:
                    # если нашли позицию для разделения
                    if split_position is not None:
                        # залогируем ее
                        split_positions.append(split_position)
                        # передвинем границу от текущего конца предложения до max_len-а
                        border = split_position + self.max_len
                        # ищем ее по новое
                        split_position = None
                    # если не нашли
                    else:
                        # используем альтернативную позицию
                        split_positions.append(split_position_alt)
                        # по ней передвигаем до макс лена
                        border = split_position_alt + self.max_len
                        split_position_alt = None
            # сохраняем полученное предложение
            split_positions.append(sequence_len)
            # для каждой разбитой подпоследовательности
            for i in range(len(split_positions) - 1):
                # добавляем спецтокены
                s_input_ids = self.add_special_tokens(
                    encoding.ids[split_positions[i]:split_positions[i + 1]],
                    'input_ids'
                )
                sequences['input_ids'].append(s_input_ids)
                # если разметка есть
                if labels is not None:
                    # добавляем спецтокены для аттеншен маск
                    s_attention_mask = self.add_special_tokens(
                        encoding.attention_mask[split_positions[i]:split_positions[i + 1]],
                        'attention_mask'
                    )
                    sequences['attention_mask'].append(s_attention_mask)
                    # и для лейблов
                    s_labels = self.add_special_tokens(
                        labels[split_positions[i]:split_positions[i + 1]],
                        'labels'
                    )
                    sequences['labels'].append(s_labels)
 
        return sequences

    # обработка обучающей, валидационной и тестовой
    # train_path, valid_path, test_path: пути к файлам с выборками
    # производит токенизацию каждой выборки
    def preprocess(self, train_path, valid_path, test_path):
        # пути к выборкам
        datasets_paths = {
            'train': train_path,
            'valid': valid_path,
            'test': test_path
        }
 
        sequences = dict()
        # для каждой части
        for sample_name in ['train', 'valid', 'test']:
            
            sequences[sample_name] = {t: [] for t in self.tensor_names}
            # токенизируем все тексты
            encodings, labels = self.tokenize_texts(datasets_paths[sample_name])

            # определеяется длина самой длинной последовательности, но не более 511
            if sample_name == 'train':
                self.max_len = min(self.max_len, max(map(lambda e: len(e.ids), encodings)))
            # итерируемся по всем лейблам
            for i in range(len(labels)):
                # разбиваем на подпоследовательности
                sequence = self.cut_long_sequence(encodings[i], labels[i])
                # добавляем коротки последовательности другими, чтоб формировать батчи большего размера
                for t in self.tensor_names:
                    sequences[sample_name][t].extend(sequence[t])
            # конвертируем в торч тензор
            for t in self.tensor_names:
                sequences[sample_name][t] = torch.tensor(sequences[sample_name][t], dtype=torch.long)
 
        return sequences

    # функция для инференса
    # инициализирует препроцессор словарями отображения меток
    def set_id_to_label(self, id_to_label):
        self.id_to_label = id_to_label
        self.label_to_id = {l: i for i, l in id_to_label.items()}

    # функция для инференса
    # устанавливает max len
    def set_max_len(self, max_len):
        self.max_len = max_len

### Trainer

In [ ]:
import numpy as np
import torch
from tqdm.auto import tqdm
from torch.nn.utils import clip_grad_norm_
from transformers import BertConfig, BertForTokenClassification, AdamW, get_scheduler
from seqeval.metrics import classification_report

torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
 
class Trainer:
    def __init__(
        self,
        preprocessor, # экземпляр класса препроцессора
        checkpoint_path, # путь к чекпоинту модели
        bert_config_path, # путь к конфигу берта
        bert_weights_path, # путь к весам берта
        epochs=10, # количество эпох
    ):
        # инициализируем все переданное в переменные
        self.preprocessor = preprocessor
        self.config = BertConfig.from_pretrained(bert_config_path, num_labels=len(self.preprocessor.id_to_label))
        self.model = BertForTokenClassification.from_pretrained(bert_weights_path, config=self.config)
        self.epochs = epochs
        self.checkpoint_path = checkpoint_path
        # устанавливаем устрйоство для дообучения
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # переносим модель на устрйоство
        self.model.to(self.device)

    # функция для валидации и тестирования
    # dataloader: генератор батчей
    # all_metrics: False - возвращаем loss, True - возвращаем loss и метрики
    def evaluate(self, dataloader, all_metrics=False):
        # отключаем дропауты и накопления в батчнормах
        self.model.eval()
 
        eval_loss = 0
        true_labels = []
        pred_labels = []
        # итерируемся по батчам
        for batch in dataloader:
            # переносим их на устройство
            batch = {k: v.to(self.device) for k, v in batch.items()}
            # получаем прогноз
            with torch.no_grad():
                outputs = self.model(**batch)
            # и логиты
            logits = outputs.logits.detach().cpu().numpy()
            # логируем вал. лосс
            eval_loss += outputs.loss.item()
            # конвертируем лейблы в numpy
            labels = batch['labels'].to('cpu').numpy()
            # логируем прогнозы для рассчета метрик
            if all_metrics:
                true_labels.extend(labels)
                pred_labels.extend([list(p) for p in np.argmax(logits, axis=2)])
        # валидационный лосс
        eval_loss = eval_loss / len(dataloader)
        # рассчитывать ли метрики качества или просто сравнить модели по val loss
        if all_metrics:
 
            true_labels_names = []
            pred_labels_names = []
            # итерируемся по разметке и предсказанным лейблам
            for tls, pls in zip(true_labels, pred_labels):
 
                t_labels_names = []
                p_labels_names = []
                # логируем пары, если это не pad токен
                for t, p in zip (tls, pls):
 
                    if self.preprocessor.id_to_label[t] != '[PAD]':
                        t_labels_names.append(self.preprocessor.id_to_label[t])
                        p_labels_names.append(self.preprocessor.id_to_label[p])
 
                true_labels_names.append(t_labels_names)
                pred_labels_names.append(p_labels_names)
            # рассчитываем метрики качества
            metrics = classification_report(true_labels_names, pred_labels_names, output_dict=True)
            for k in metrics:
                del metrics[k]['support']
            # логируем валидационный лосс
            metrics['eval_loss'] = eval_loss
        # иначе просто логируем лосс
        else:
            metrics = dict()
            metrics['eval_loss'] = eval_loss
 
        return metrics

    # функция обучение модели
    # train_dataloader, valid_dataloader - генераторы батчей
    def fit(self, train_dataloader, valid_dataloader):
        # оптимизатор
        optimizer = AdamW(self.model.parameters(), lr=5e-5)
        max_grad_norm = 1.0
        num_training_steps = self.epochs * len(train_dataloader)
        # шедулер для линейного разгона
        scheduler = get_scheduler(
            'linear',
            optimizer=optimizer,
            num_warmup_steps=0,
            num_training_steps=num_training_steps
        )
 
        self.train_losses = []
        self.valid_losses = []
        best_loss = np.inf
        # красивый прогресс бар
        with tqdm(range(num_training_steps)) as progress_bar:
            # итерация по эпохам
            for epoch in range(self.epochs):
                # включаем дропауты и батчнормы
                self.model.train()
 
                train_loss = 0
                # идем по загрузчику
                for batch in train_dataloader:
                    # переносим батч на гпу
                    batch = {k: v.to(self.device) for k, v in batch.items()}
                    # получаем прогнозы
                    outputs = self.model(**batch)
                    # находим градиент в точке
                    loss = outputs.loss
                    loss.backward()
                    # логируем лосс
                    train_loss += loss.item()
                    # клипаем градиент
                    clip_grad_norm_(parameters=self.model.parameters(), max_norm=max_grad_norm)
                    # обновляем параметры
                    optimizer.step()
                    scheduler.step()
                    # обнуляем градиент
                    self.model.zero_grad()
                    # обновляем прогессбар
                    progress_bar.update(1)
                # лосс на треине
                train_loss = train_loss / num_training_steps
                # лосс на валидации
                eval_loss = self.evaluate(valid_dataloader)['eval_loss']
                # логируем loss-ы
                self.train_losses.append(train_loss)
                self.valid_losses.append(eval_loss)
                # если валидационный лучше текущего - сохраняем чекпоинт
                if eval_loss < best_loss:
                    best_loss = eval_loss
                    torch.save(
                        {
                            'model': self.model.state_dict(),
                            'id_to_label': self.preprocessor.id_to_label,
                            'max_len': self.preprocessor.max_len,
                            'batch_size': train_dataloader.batch_size
                        },
                        self.checkpoint_path
                    )
        # в конце обучения подгружаем лучший чекпоинт
        checkpoint = torch.load(self.checkpoint_path)
        # из него берем модель
        self.model = BertForTokenClassification.from_pretrained(pretrained_model_name_or_path=None, state_dict=checkpoint['model'], config=self.config)
        self.model.to(self.device)

# Predictor

In [ ]:
import numpy as np
import torch
from transformers import BertConfig, BertForTokenClassification
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
 
# класс для загрузки модели и ковертации BIO -> jsonlines
class Predictor:
    def __init__(
        self,
        preprocessor, # экземпляр предобработчика
        checkpoint_path, # путь к чекпоинту
        bert_config_path, # путь к конфигу
    ):
        # подгружаем конфиг и чекпоинт инициализируем им все параметры
        self.preprocessor = preprocessor
        checkpoint = torch.load(checkpoint_path)
        self.preprocessor.set_id_to_label(checkpoint['id_to_label'])
        self.preprocessor.set_max_len(checkpoint['max_len'])
        self.batch_size = checkpoint['batch_size']
        self.config = BertConfig.from_pretrained(bert_config_path, num_labels=len(self.preprocessor.id_to_label))
        self.model = BertForTokenClassification.from_pretrained(pretrained_model_name_or_path=None, state_dict=checkpoint['model'], config=self.config)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device);

    # прогноз моделью
    # принимает на вход строку text, возвращает список именнованных сущностей
    # каждая сущность словарь с типом сущности и ее началом и концом
    def predict(self, text):
        # токенизация тектов
        encoding = self.preprocessor.tokenizer(text, add_special_tokens=False)[0]
        # разделение на части
        sequences = self.preprocessor.cut_long_sequence(encoding)['input_ids']
        # id-ники слов
        input_ids = torch.tensor(sequences)
        # создаем датасет
        dataset = TensorDataset(input_ids)
        # создаем загрузчик
        dataloader = DataLoader(dataset, batch_size=self.batch_size, sampler=SequentialSampler(dataset))
 
        labels = []
        # идем по всем батчам
        for batch in dataloader:
            batch = batch[0]
            # получаем прогнозы
            with torch.no_grad():
                outputs = self.model(batch.to(self.device))
            labels_array = np.argmax(outputs.logits.detach().cpu().numpy(), axis=2)
            # добавляем лейблы не из служебных токенов
            for i in range(len(labels_array)):
                l = labels_array[i][(batch[i] != self.preprocessor.tokenizer.pad_token_id) & (batch[i] != self.preprocessor.tokenizer.cls_token_id)]
                labels.extend(l)
 
        entities = []
        # конвертируем айдишники выхода в лейблы
        for i in range(len(encoding.tokens)):
            token_labels = set(self.preprocessor.id_to_label[labels[i]].split('|'))
            for label in token_labels:
                # если метка слова - продолждение, т.е. является первой меткой сущности,
                # то по номеру соответствующего ей входного токена с помощью token_to_chars
                #  объекта encoding определяется начальная и конечная позиция токена
                if label[0] == 'B':
                    start, end = encoding.token_to_chars(i)
                    entity = {
                        'token_num': i,
                        'label': label[2:],
                        'start': start,
                        'end': end
                    }
                    entities.append(entity)
        '''
        Для каждой сущности из списка entities просматриваются токены, 
        начиная от позиции, следующей за token_num, до конца последовательности. 
        Аналогичным образом для каждого токена создаётся список соответствующих 
        ему меток token_labels. Если название сущности есть в списке token_labels, 
        то с помощью функции token_to_chars вычисляется конечная позиция данного 
        токена и обновляется конечная позиция сущности. 
        Просмотр токенов заканчивается, как только название сущности не будет 
        обнаружено в списке token_labels.
        '''
        for e in entities:
            for i in range(e['token_num'] + 1, len(encoding.tokens)):
                token_labels = set(self.preprocessor.id_to_label[labels[i]].split('|'))
                label = e['label']
                label = f'I-{label}'
                if label in token_labels:
                    end = encoding.token_to_chars(i)[1]
                    e['end'] = end
                else:
                    break
            del e['token_num']
 
        return entities

![](https://habrastorage.org/r/w1560/getpro/habr/upload_files/89d/04f/ca2/89d04fca27aa7840436b5d471f75d390.png)

![](https://habrastorage.org/getpro/habr/upload_files/670/87e/eac/67087eeac7f8c8093542afd1494865d0.JPG)

### BatchSizeSelector

In [ ]:
import psutil
import torch
from transformers import BertConfig, BertForTokenClassification
 
 
torch.backends.cudnn.deterministic = True
 
'''
Метод автоматического подбора размера батча по исключению «CUDA out of memory»
'''
class BatchSizeSelector:
 
    def __init__(
        self,
        preprocessor,
        bert_config_path,
        bert_weights_path,
    ):
        self.preprocessor = preprocessor # объект класса, подготавливающего текст
        self.config = BertConfig.from_pretrained(bert_config_path, num_labels=len(self.preprocessor.id_to_label))
        self.model = BertForTokenClassification.from_pretrained(bert_weights_path, config=self.config)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device);
 
 
    def generate_batch(self, size):
        # заполним рандомными значениями
        torch.manual_seed(0)
        batch = dict()
        batch['attention_mask'] = torch.ones(size=size, dtype=torch.long)
        batch['input_ids'] = torch.randint(low=1, high=self.preprocessor.tokenizer.vocab_size, size=size, dtype=torch.long)
        batch['labels'] = torch.randint(low=0, high=len(self.preprocessor.id_to_label), size=size, dtype=torch.long)
 
        return batch
 
 
    def try_size_cuda(self, size):
        # переберем разные размеры батча
        try:
            batch = self.generate_batch(size)
            batch = {k: v.to(self.device) for k, v in batch.items()}
            outputs = self.model(**batch)
            return True
        except RuntimeError as e:
            if len(e.args) == 1 and 'CUDA out of memory.' in e.args[0]:
                return False
            else:
                raise e
 
 
    def try_size_cpu(self, size):
        # реализация для перебора на cpu
        batch = self.generate_batch(size)
        batch = {k: v.to(self.device) for k, v in batch.items()}
        outputs = self.model(**batch)
        vm = psutil.virtual_memory()
        if vm.percent >= 85:
            return False
        else:
            return True
 
 
    def get_optimal_size(self):
        # начинаем с 1 и идем по степени двойки
        batch_size = 1
        sequence_len = self.preprocessor.max_len + 1
        self.model.train()
 
        while True:
            size = (batch_size, sequence_len)
 
            if self.device.type == 'cuda':
                success = self.try_size_cuda(size)
                torch.cuda.empty_cache()
            else:
                success = self.try_size_cpu(size)
 
            if success:
                batch_size *= 2
            else:
                batch_size //= 2
                return batch_size
 
 
    def release_memory(self):
        # на каждой итерации чистим память
        device_type = self.device.type
 
        del self.preprocessor
        del self.config
        del self.model
        del self.device
 
        if device_type == 'cuda':
            torch.cuda.empty_cache()

## AutoNER

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizerFast
import json
import torch


class AutoNER:
    @staticmethod
    def fit(
        checkpoint_path,
        report_path,
        bert_vocab_path,
        bert_config_path,
        bert_weights_path,
        train_path,
        valid_path,
        test_path,
        batch_size='auto',
        epochs=10
    ):
        # инициализируем модель
        tokenizer = BertTokenizerFast(vocab_file=bert_vocab_path, do_lower_case=False)
        preprocessor = Preprocessor(tokenizer)
        # подготавливаем данные
        sequences = preprocessor.preprocess(train_path, valid_path, test_path)
 
        # подбираем размер батча
        if batch_size == 'auto':
            batch_size_selector = BatchSizeSelector(preprocessor, bert_config_path, bert_weights_path)
            bs = batch_size_selector.get_optimal_size()
            batch_size_selector.release_memory()
        else:
            bs = batch_size

        # разбиваем датасет
        train_dataset = TokenizedDataset(sequences['train'])
        valid_dataset = TokenizedDataset(sequences['valid'])
        test_dataset = TokenizedDataset(sequences['test'])
 
        # создаем загрузчики
        train_dataloader = DataLoader(train_dataset, batch_size=bs, sampler=RandomSampler(train_dataset))
        valid_dataloader = DataLoader(valid_dataset, batch_size=bs, sampler=SequentialSampler(valid_dataset))
        test_dataloader = DataLoader(test_dataset, batch_size=bs, sampler=SequentialSampler(test_dataset))
        
        # обучаем
        trainer = Trainer(
            preprocessor,
            checkpoint_path,
            bert_config_path,
            bert_weights_path,
            epochs
        )
        trainer.fit(train_dataloader, valid_dataloader)

        # рассчитываем метрики качества
        train_metrics = trainer.evaluate(train_dataloader, all_metrics=True)
        valid_metrics = trainer.evaluate(valid_dataloader, all_metrics=True)
        test_metrics = trainer.evaluate(test_dataloader, all_metrics=True)

        # логируем результат
        report = {
            'loss': {
                'train': trainer.train_losses,
                'valid': trainer.valid_losses
            },
            'metrics': {
                'train': train_metrics,
                'valid': valid_metrics,
                'test': test_metrics
            }
        }
 
        # сохраняем результаты
        with open(report_path, 'w') as f:
            json.dump(report, f)
        # чистим память
        del trainer
        torch.cuda.empty_cache()

    @staticmethod
    def from_pretrained(
        checkpoint_path,
        bert_vocab_path,
        bert_config_path,
    ):
        # подгружаем предобученную с помощью экземпляра predictor-а
        tokenizer = BertTokenizerFast(vocab_file=bert_vocab_path, do_lower_case=False)
        preprocessor = Preprocessor(tokenizer)
        predictor = Predictor(preprocessor, checkpoint_path, bert_config_path)
 
        return predictor

# Обучение

[Зеркало](https://dl.uploadgram.me/62e1548c7ed29h?raw)

!wget -o collections5.zip https://dl.uploadgram.me/62e1548c7ed29h?raw

In [ ]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip
!unzip collection5.zip

--2022-12-06 17:16:57--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M   916KB/s    in 2.0s    

2022-12-06 17:17:01 (916 KB/s) - ‘collection5.zip’ saved [1899530/1899530]

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5

### Подготовка Collecion5 к обучению

In [ ]:
import pandas as pd

files = []

error=0
for fname in pd.Series([e.split('.')[0] for e in os.listdir('Collection5')]).unique():
    data = {}
    try:
        with open(os.path.join('Collection5', fname + '.txt')) as f:
            # print(f.read())
            text = f.read()
        df = pd.read_csv(os.path.join('Collection5', fname + '.ann'), 
                        sep='\t', header=None, names=['num', '_', 'text', 'entities', 'start', 'end'])
        df[['entities', 'start', 'end']] = df['_'].str.split(' ', 2, expand=True)
        df[['start', 'end']] = df[['start', 'end']].astype(int)
        df.drop(columns=['_'],inplace=True)
        text = text.replace('\n', '  ')

        # сохраняем данные 
        data['text'] = text
        data['entities'] = []
        # проверка на сломанные сущности
        for i, row in df.iterrows():
            if text[row['start']:row['end']] != row['text']:
                assert Exception
                # print('row text', row['text'])
                # print('real text', text[row['start']: row['end']])
            data['entities'].append({'label': row['entities'], 
                                    'start': row['start'],
                                    'end': row['end']})
        files.append(data)
    except:
        error+=1

In [ ]:
error

1

In [ ]:
text

'На выборы ректора вуза в Чувашии охрана не пустила журналистов и полицию    21 июня в Чувашской государственной сельскохозяйственной академии избирали нового ректора. День голосования обернулся очередным скандалом с выборными технологиями, как предрекали накануне некоторые политики. Показательная марш-операция с форсированной сменой ректора превратила учебную площадку в центрально-политический избирательный штаб, а сам процесс принял настолько уродливые формы, что даже терпимым ко многому и привыкшим к разному журналистам пришлось на этот раз вызывать полицию, передаёт корреспондент ИА REGNUM.    Выборная хроника, напоминающая больше военную, начинает отсчёт в декабре 2012 года, когда устранили предыдущего ректора 51-летнего Николая Васильева, проработавшего менее года: он был внезапно отправлен в отставку по инициативе учредителя - Минсельхоза России (министерство возглавляет бывший президент Чувашии Николай Федоров). Одновременно состоялось представление 58-летней Людмилы Линик: быв

In [ ]:
df.head()

,num,text,entities,start,end
0,T1,Чувашии,LOC,25,32
1,T2,Чувашской государственной сельскохозяйственной...,ORG,86,141
2,T3,ИА REGNUM,MEDIA,590,599
3,T4,Николая Васильева,PER,735,752
4,T5,Минсельхоза,ORG,845,856


In [ ]:
import json
with open('train.jsonlines', 'w', encoding='utf8') as fp:
    fp.write('\n'.join(json.dumps(i, ensure_ascii=False) for i in files[:800]))

import json
with open('val.jsonlines', 'w', encoding='utf8') as fp:
    fp.write('\n'.join(json.dumps(i, ensure_ascii=False) for i in files[800:850]))
    
import json
with open('test.jsonlines', 'w', encoding='utf8') as fp:
    fp.write('\n'.join(json.dumps(i, ensure_ascii=False) for i in files[850:]))
    

In [ ]:
AutoNER.fit(
    checkpoint_path = './model_checkpoint.pth',
    report_path = './report.json',
    bert_vocab_path = './sbert_large_nlu_ru/vocab.txt',
    bert_config_path = './sbert_large_nlu_ru/config.json',
    bert_weights_path = './sbert_large_nlu_ru/pytorch_model.bin',
    train_path = './train.jsonlines',
    valid_path = './val.jsonlines',
    test_path = './test.jsonlines',
    batch_size='auto',
    epochs=10,
)

error with entity: {'label': 'PER', 'start': 2311, 'end': 2318}
error with entity: {'label': 'LOC', 'start': 83, 'end': 91}
error with entity: {'label': 'LOC', 'start': 83, 'end': 91}


Some weights of BertForTokenClassification were not initialized from the model checkpoint at ./sbert_large_nlu_ru/pytorch_model.bin and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at ./sbert_large_nlu_ru/pytorch_model.bin and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/2220 [00:00<?, ?it/s]

TypeError: ignored

In [ ]:
predictor = AutoNER.from_pretrained(
    checkpoint_path = './model_checkpoint.pth',
    bert_vocab_path = './sbert_large_nlu_ru/vocab.txt',
    bert_config_path = './sbert_large_nlu_ru/config.json'
)

In [ ]:
test = '''
ООО Рога и копыта, договор подписан Василием Петровичем в Омске.
'''

entities = predictor.predict(test)

for entity in entities:
    print(f"Entity {entity['label']} | text: {test[entity['start']:entity['end']]}")


Entity ORG | text: ООО Рога и копыта
Entity PER | text: Василием Петровичем
Entity LOC | text: Омске
